In [28]:
print("Shivaya Namah")

Shivaya Namah


In [1]:
from pydantic_settings import BaseSettings

class Settings(BaseSettings):

    pinecone_api_key: str 
    watsonx_apikey: str  
    watsonx_url: str
    watsonx_project_id: str
    cohere_api_key: str 
    tavily_api_key: str

    class Config:
        env_file = ".env"
        extra = "ignore"

settings = Settings()
print("API Keys loaded succesfully.")

API Keys loaded succesfully.


In [30]:
from langchain_cohere import ChatCohere

llm = ChatCohere(cohere_api_key = settings.cohere_api_key, temperature = 0.7)

In [31]:
llm.invoke("Hi this is Nagarjun, you")

AIMessage(content="Hello Nagarjun! I'm Command, an AI assistant. How can I help you today? Whether it's answering questions, brainstorming ideas, or assisting with a project, I'm here to support you. What do you need?", additional_kwargs={'id': '4502739d-5fdb-48d2-8bb0-26d7b4dfbd94', 'finish_reason': 'COMPLETE', 'content': "Hello Nagarjun! I'm Command, an AI assistant. How can I help you today? Whether it's answering questions, brainstorming ideas, or assisting with a project, I'm here to support you. What do you need?", 'token_count': {'input_tokens': 502.0, 'output_tokens': 48.0}}, response_metadata={'id': '4502739d-5fdb-48d2-8bb0-26d7b4dfbd94', 'finish_reason': 'COMPLETE', 'content': "Hello Nagarjun! I'm Command, an AI assistant. How can I help you today? Whether it's answering questions, brainstorming ideas, or assisting with a project, I'm here to support you. What do you need?", 'token_count': {'input_tokens': 502.0, 'output_tokens': 48.0}}, id='run--6495ca37-c79b-4153-963f-4b8e7

In [32]:
llm.invoke("what is my current location?")

AIMessage(content="I'm unable to determine your current location as I don't have access to your device's GPS or location services. My purpose is to provide information and assist with tasks, but I don't have the capability to track or access personal data like your location.\n\nIf you'd like to know your current location, I recommend checking your device's settings or using a mapping application like Google Maps or Apple Maps. These tools can provide you with accurate and up-to-date information about your whereabouts.\n\nIf you have any other questions or need assistance with something else, feel free to ask!", additional_kwargs={'id': '5805e5eb-4684-4962-ad5c-23e0a38972e9', 'finish_reason': 'COMPLETE', 'content': "I'm unable to determine your current location as I don't have access to your device's GPS or location services. My purpose is to provide information and assist with tasks, but I don't have the capability to track or access personal data like your location.\n\nIf you'd like t

# Initializing the agent

In [69]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_community.tools import TavilySearchResults


search_tool = TavilySearchResults(search_depth="basic", tavily_api_key = settings.tavily_api_key)
search_tool

TavilySearchResults(search_depth='basic', api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))

In [77]:
def get_current_date(*args, **kwargs):
    """Returns the current Date."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %I:%M:%S, %p")

get_current_date()

'2025-07-14 03:54:30, PM'

In [78]:
tools: list[Tool] = [
    search_tool,
    Tool(
        name = "current_datetime",
        func = get_current_date,
        description = "Use when you need to know the current date and time."
    )
]
tools

[TavilySearchResults(search_depth='basic', api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 Tool(name='current_datetime', description='Use when you need to know the current date and time.', func=<function get_current_date at 0x0000015568CF2480>)]

In [79]:
agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)
agent

AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\ntavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\ncurrent_datetime(*args, **kwargs) - Use when you need to know the current date and time.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [tavily_search_results_json, current_datetime]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought

In [75]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
current_datetime() - Use when you need to know the current date and time.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [tavily_search_results_json, current_datetime]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [14]:
agent.invoke("give me the funny tweet about today weather in chennai.")



> Entering new AgentExecutor chain...
Thought: To find a funny tweet about today's weather in Chennai, I need to first find out what the weather is like in Chennai today. Then, I can search for tweets related to that weather condition.

Action: tavily_search_results_json
Action Input: "Chennai weather today"O
Observation: [{'title': 'Weather in Chennai', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Chennai', 'region': 'Tamil Nadu', 'country': 'India', 'lat': 13.0833, 'lon': 80.2833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1752485458, 'localtime': '2025-07-14 15:00'}, 'current': {'last_updated_epoch': 1752485400, 'last_updated': '2025-07-14 15:00', 'temp_c': 33.6, 'temp_f': 92.5, 'is_day': 1, 'condition': {'text': 'Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/119.png', 'code': 1006}, 'wind_mph': 8.7, 'wind_kph': 14.0, 'wind_degree': 125, 'wind_dir': 'SE', 'pressure_mb': 1002.0, 'pressure_in': 29.58, 'precip_mm': 0.0, 'precip_in': 0.0, 'humid

{'input': 'give me the funny tweet about today weather in chennai.',
 'output': 'Unfortunately, I couldn\'t find a specific funny tweet about today\'s weather in Chennai. However, given that it\'s mostly cloudy with temperatures reaching up to 37°C, here’s a humorous take: "Chennai\'s weather today is like a bad date – cloudy, hot, and leaving you wondering when it’s going to clear up!"'}

In [20]:
agent.invoke("Today weather in chennai.")



> Entering new AgentExecutor chain...
Question: Today weather in chennai.  
Thought: To answer this question, I need to find the current weather conditions in Chennai. I will use the tavily_search_results_json tool to search for "today weather in Chennai."  
Action: tavily_search_results_json  
Action Input: today weather in ChennaiO
Observation: [{'title': 'Weather in Chennai', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Chennai', 'region': 'Tamil Nadu', 'country': 'India', 'lat': 13.0833, 'lon': 80.2833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1752485814, 'localtime': '2025-07-14 15:06'}, 'current': {'last_updated_epoch': 1752485400, 'last_updated': '2025-07-14 15:00', 'temp_c': 33.6, 'temp_f': 92.5, 'is_day': 1, 'condition': {'text': 'Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/119.png', 'code': 1006}, 'wind_mph': 8.7, 'wind_kph': 14.0, 'wind_degree': 125, 'wind_dir': 'SE', 'pressure_mb': 1002.0, 'pressure_in': 29.58, 'precip_mm': 0.0,

{'input': 'Today weather in chennai.',
 'output': 'The weather in Chennai today is cloudy with a temperature of 33.6°C (92.5°F). The conditions include a humidity level of 52%, wind speed of 14.0 kph from the southeast, and no precipitation. It feels like 38.6°C (101.5°F) due to the heat index.'}

In [36]:
agent.invoke("Write a funny tweet about today weather in chennai.")



> Entering new AgentExecutor chain...
Question: Write a funny tweet about today's weather in Chennai.  
Thought: To write a funny tweet about today's weather in Chennai, I need to first find out what the weather is like there today.  
Action: tavily_search_results_json  
Action Input: "Chennai weather today"O
Observation: [{'title': 'Weather in Chennai', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Chennai', 'region': 'Tamil Nadu', 'country': 'India', 'lat': 13.0833, 'lon': 80.2833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1752486413, 'localtime': '2025-07-14 15:16'}, 'current': {'last_updated_epoch': 1752486300, 'last_updated': '2025-07-14 15:15', 'temp_c': 33.6, 'temp_f': 92.5, 'is_day': 1, 'condition': {'text': 'Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/119.png', 'code': 1006}, 'wind_mph': 8.7, 'wind_kph': 14.0, 'wind_degree': 125, 'wind_dir': 'SE', 'pressure_mb': 1002.0, 'pressure_in': 29.58, 'precip_mm': 0.0, 'precip_in': 0.0, 'humid

{'input': 'Write a funny tweet about today weather in chennai.',
 'output': '"Chennai weather today: 37°C, feels like a sauna with a side of monsoon. Bring your umbrella and a portable AC, because Mother Nature’s got her settings on \'tropical torture\' mode. #ChennaiWeather #HotAndBothered"'}

In [81]:
agent.invoke("What is the todays date and time?")



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to use the current_datetime tool to get the current date and time.

Action: current_datetime
Action Input: NoneO
Observation: 2025-07-14 03:56:26, PM
Thought:Thought: I now know the final answer.

Final Answer: Today's date and time is 2025-07-14 03:56:26 PM.

> Finished chain.


{'input': 'What is the todays date and time?',
 'output': "Today's date and time is 2025-07-14 03:56:26 PM."}